<img src="../../img/python-logo-no-text.svg"
     style="display:block;margin:auto;width:10%"/>
<br>
<div style="text-align:center; font-size:200%;">
  <b>SOLID: Single Responsibility Principle</b>
</div>
<br/>
<div style="text-align:center;">Dr. Matthias Hölzl</div>
<br/>
<div style="text-align:center;">module_280_solid/topic_120_a3_solid_srp</div>


# SOLID: Single Responsibility Principle

What do you think of the following code?

In [ ]:
from typing import NamedTuple

In [ ]:
class Point2D(NamedTuple):
    x: float
    y: float

    def move(self, dx: float, dy: float) -> "Point2D":
        return Point2D(self.x + dx, self.y + dy)

In [ ]:
class Figure2d(NamedTuple):
    pivot: Point2D
    sprite: list[int]
    health: float
    health_bar: object  # In reality a widget...
    database: object  # The connection to the DB

    def move(self, dx: float, dy: float) -> None:
        ...

    def is_player_avatar(self) -> bool:
        # Check whether the figure is the player's avatar...
        return True

    def update_gui(self):
        # Update the health bar widget.
        ...

    def save_to_db(self) -> None:
        # Write the figure data to the database.
        ...

In [ ]:
class GameEditor:
    def edit_figure(self, figure: Figure2d):
        # Edit the figure...
        figure.save_to_db()

In [ ]:
class Game(NamedTuple):
    player_avatar: Figure2d

    def run_game_loop(self):
        # Do lots of things...
        self.player_avatar.update_gui()


## Single-Responsibility Principle

- For each class there should only be a single reason to change
- The name is not quite correct: SRP does not state that each class may only
  have a single responsibility


## Single Responsibility?

<img src="img/book_01.png"
     style="display:block;margin:auto;width:35%"/>



## Violation of SRP

<img src="img/book_02.png"
     style="display:block;margin:auto;width:60%"/>


In [ ]:
from dataclasses import dataclass  # noqa: #402

In [ ]:
@dataclass
class Book:
    title: str
    author: str
    pages: int

    def print(self):
        # Lots of code that handles the printer
        print("Printing to printer.")

    def save(self):
        # Lots of code that handles the database
        print("Saving to database.")


## Resolution of the SRP violation (version 1)

<img src="img/book_resolution_1.png"
     style="display:block;margin:auto;width:50%"/>


## Resolution of the SRP violation (version 2)

<img src="img/book_resolution_2.png"
     style="display:block;margin:auto;width:80%"/>


## Comparison

<div>
<img src="img/book_resolution_1.png"
     style="float:left;padding:5px;width:40%"/>
<img src="img/book_resolution_2.png"
     style="float:right;padding:5px;width:50%"/>
</div>


## Mini workshop: Employee

You have the following implementation of a personnel management system that
contains several SRP violations. Implement a version that no longer has them.

<img src="img/employee_01.png"
     style="display:block;margin:auto;width:40%"/>

In [ ]:
from dataclasses import dataclass  # noqa: E402
from enum import IntEnum  # noqa: E402

In [ ]:
class EmployeeType(IntEnum):
    REGULAR = 0
    HOURED = 1
    COMMISSIONED = 2

In [ ]:
@dataclass
class Project:
    name: str
    assets: float

In [ ]:
@dataclass
class AugurDatabase:
    def start_transaction(self):
        pass

    def commit_transaction(self):
        pass

    def rollback_transaction(self):
        pass

    def store_field(self, obj_id, name, value):
        pass

In [ ]:
@dataclass
class Employee:
    id: int
    name: str
    salary: float
    overtime: int
    employee_type: EmployeeType
    project: Project
    database: AugurDatabase

    def calculate_pay(self) -> float:
        if self.employee_type == EmployeeType.REGULAR:
            return self.salary + 60.0 * self.overtime
        elif self.employee_type == EmployeeType.COMMISSIONED:
            return self.project.assets * 0.1
        elif self.employee_type == EmployeeType.HOURED:
            return 50.0 * self.overtime
        raise ValueError(f"{self.employee_type} is not valid.")

    def report_hours(self) -> int:
        if self.employee_type == EmployeeType.REGULAR:
            return 40 + self.overtime
        elif self.employee_type == EmployeeType.COMMISSIONED:
            # Commissioned employees always work 40 hours
            return 40
        elif self.employee_type == EmployeeType.HOURED:
            # We use overtime for the billed hours
            return self.overtime
        raise ValueError(f"{self.employee_type} is not valid.")

    def print_report(self) -> None:
        print(f"{self.name} worked {self.report_hours()} hours.")

    def save_employee(self) -> None:
        self.database.start_transaction()
        self.database.store_field(self.id, "name", self.name)
        self.database.store_field(self.id, "salary", self.salary)
        self.database.store_field(self.id, "overtime", self.overtime)
        self.database.store_field(self.id, "employee_type", self.employee_type)
        self.database.store_field(self.id, "project", self.project)
        self.database.store_field(self.id, "database", self.database)
        self.database.commit_transaction()